# Parameter Efficient Finetuning (PEFT) using Low Rank Adapters (LoRA) techniques

## Goals
1. Write a PyTorch training loop implementing LoRA via PEFT
2. Tokenize multiple sclerosis data first
3. Parallelize LoRA on all GPUs if possible

## Steps
1. Integrate HuggingFace's PEFT into scGPT to perform finetuning
2. Implementation will use HuggingFace's scGPT implementation from Therapeutic Commons - https://huggingface.co/tdc/scGPT
3. Test dataset - M.S. dataset (since there is a benchmark)

Requirements from HuggingFace
- transformers 
- accelerate 
- evaluate
- datasets 
- peft
- loralib
- PyTDC

In [41]:
# HF imports 
import transformers
import accelerate
import peft
# from peft import get_peft_model, LoraConfig, TaskType
import datasets

# TDC Imports
from tdc.multi_pred.anndata_dataset import DataLoader
from tdc import tdc_hf_interface
from tdc.model_server.tokenizers.scgpt import scGPTTokenizer
from tdc.model_server.models import scgpt

import torch
import numpy as np
import scanpy as sc



print(f"Transformers version: {transformers.__version__}")
print(f"Accelerate version: {accelerate.__version__}")
print(f"PEFT version: {peft.__version__}")
print(f"Datasets version: {datasets.__version__}")
# print(f"TDC version: {tdc.__version__")

Transformers version: 4.50.3
Accelerate version: 0.33.0
PEFT version: 0.15.1
Datasets version: 2.19.2


# Step 1: Load data

1. Load raw counts from training and test dataset
2. Follow steps for normalization, tokenization, and embedding



In [42]:
data_path = "../data/lora_test/"

adata = sc.read_h5ad(data_path+"c_data.h5ad")

# del adata.uns["log1p"]

## ALREADY DONE FOR MULTIPLE SCLEROSIS DATA

# Normalize and log1p — don't do HVG or batch correction
# sc.pp.normalize_total(adata, target_sum=1e4)
# sc.pp.log1p(adata)

In [52]:
# Load pretrained scGPT model
scgpt = tdc_hf_interface("scGPT")
base_model = scgpt.load()

# Load tokenizer
tokenizer = scGPTTokenizer()

gene_names = adata.var["gene_name"].to_numpy()  # Convert gene_names to numpy array

tokenized_data = tokenizer.tokenize_cell_vectors(adata.X.toarray(), gene_names)

Found local copy...


In [44]:
print(base_model)

ScGPTModel(
  (gene_encoder): ModuleDict(
    (embedding): Embedding(60697, 512, padding_idx=0)
    (enc_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
  )
  (value_encoder): ModuleDict(
    (linear1): Linear(in_features=1, out_features=512, bias=True)
    (linear2): Linear(in_features=512, out_features=512, bias=True)
    (norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.0, inplace=False)
  )
  (transformer): TransformerEncoder(
    (layers): ModuleList(
      (0-11): 12 x TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=512, out_features=512, bias=True)
        )
        (linear1): Linear(in_features=512, out_features=512, bias=True)
        (dropout): Dropout(p=0.0, inplace=False)
        (linear2): Linear(in_features=512, out_features=512, bias=True)
        (norm1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((

In [45]:
mask = torch.tensor([x != 0 for x in tokenized_data[0][1]],
                    dtype=torch.bool)
# Extract first embedding
first_embed = base_model(tokenized_data[0][0],
                    tokenized_data[0][1],
                    attention_mask=mask)

In [51]:
# Wrap with LoRA (only Q and V projections usually)
lora_config = peft.LoraConfig(
    r=4,
    lora_alpha=16,
    target_modules=["self_attn"],  # You may need to adapt based on scGPT architecture
    lora_dropout=0.1,
    bias="none",
    task_type=peft.TaskType.FEATURE_EXTRACTION
)
model = peft.get_peft_model(base_model, lora_config)
model.print_trainable_parameters()

trainable params: 147,456 || all params: 50,951,681 || trainable%: 0.2894


In [29]:
print(model)

PeftModelForFeatureExtraction(
  (base_model): LoraModel(
    (model): ScGPTModel(
      (gene_encoder): ModuleDict(
        (embedding): Embedding(60697, 512, padding_idx=0)
        (enc_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      )
      (value_encoder): ModuleDict(
        (linear1): Linear(in_features=1, out_features=512, bias=True)
        (linear2): Linear(in_features=512, out_features=512, bias=True)
        (norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (dropout): Dropout(p=0.0, inplace=False)
      )
      (transformer): TransformerEncoder(
        (layers): ModuleList(
          (0-11): 12 x TransformerEncoderLayer(
            (self_attn): lora.MultiheadAttention(
              (base_layer): MultiheadAttention(
                (out_proj): lora.Linear(
                  (base_layer): NonDynamicallyQuantizableLinear(in_features=512, out_features=512, bias=True)
                  (lora_dropout): ModuleDict(
                    (defa

In [36]:
import torch.nn as nn

class CellTypeClassifier(nn.Module):
    def __init__(self, base_model, hidden_dim, num_classes):
        super().__init__()
        self.base_model = base_model
        self.classifier = nn.Linear(hidden_dim, num_classes)

    def forward(self, gene_ids, expr_vals, attention_mask=None):
        output = self.base_model.model(
            input_ids=gene_ids,
            values=expr_vals,
            attention_mask=attention_mask,
            output_cell_emb=True  # 🔑 make sure this is True!
        )

        # Extract the pooled cell embedding
        pooled = output["cell_emb"]  # shape: [batch_size, hidden_dim]
        return self.classifier(pooled)
    
from sklearn.preprocessing import LabelEncoder
import torch

# Encode your cell types
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(adata.obs["celltype"])
labels = torch.tensor(labels)

clf_model = CellTypeClassifier(model, hidden_dim=base_model.config.d_hid, num_classes=len(label_encoder.classes_))

In [37]:
next(clf_model.parameters()).device

device(type='cpu')

In [38]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
clf_model = clf_model.to(device)
next(clf_model.parameters()).device

device(type='cuda', index=0)

In [39]:
import torch.optim as optim
import torch.nn as nn

optimizer = optim.Adam(clf_model.parameters(), lr=1e-4)
loss_fn = nn.CrossEntropyLoss()

clf_model.train()
for epoch in range(5):
    total_loss = 0
    for i, (gene_ids, expr_vals) in enumerate(tokenized_data):
        mask = expr_vals != 0
        label = labels[i].unsqueeze(0).to(device)

        optimizer.zero_grad()
        pred = clf_model(
            gene_ids.unsqueeze(0).to(device),
            expr_vals.unsqueeze(0).to(device),
            attention_mask=mask.unsqueeze(0).to(device)
        )
        # print("Raw logits:", pred)
        # print("Logits shape:", pred.shape)
        # print("Max logit:", pred.max().item())
        # print("Min logit:", pred.min().item())
        loss = loss_fn(pred, label)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
    avg_loss = total_loss / len(tokenized_data)
    print(f"Epoch {epoch+1} average loss: {avg_loss:.4f}")

Epoch 1 average loss: 1.9547
Epoch 2 average loss: 1.9482


KeyboardInterrupt: 

In [40]:
adata

AnnData object with n_obs × n_vars = 13468 × 3000
    obs: 'Sample Characteristic[organism]', 'Sample Characteristic Ontology Term[organism]', 'Sample Characteristic[individual]', 'Sample Characteristic Ontology Term[individual]', 'Sample Characteristic[sex]', 'Sample Characteristic Ontology Term[sex]', 'Sample Characteristic[age]', 'Sample Characteristic Ontology Term[age]', 'Sample Characteristic[developmental stage]', 'Sample Characteristic Ontology Term[developmental stage]', 'Sample Characteristic[organism part]', 'Sample Characteristic Ontology Term[organism part]', 'Sample Characteristic[sampling site]', 'Sample Characteristic Ontology Term[sampling site]', 'Sample Characteristic[disease]', 'Sample Characteristic Ontology Term[disease]', 'Sample Characteristic[organism status]', 'Sample Characteristic Ontology Term[organism status]', 'Sample Characteristic[cause of death]', 'Sample Characteristic Ontology Term[cause of death]', 'Sample Characteristic[clinical history]', 'Sample 

In [35]:
print(f"Pred shape: {pred.shape}, Label shape: {label.shape}")

Pred shape: torch.Size([1, 18]), Label shape: torch.Size([1])


In [37]:
probs = torch.softmax(pred, dim=1)
print("Probabilities:", probs)

Probabilities: tensor([[0.0942, 0.0150, 0.0367, 0.1111, 0.1349, 0.0040, 0.2483, 0.1395, 0.0614,
         0.0064, 0.0040, 0.0101, 0.0182, 0.0139, 0.0033, 0.0420, 0.0025, 0.0547]],
       device='cuda:0', grad_fn=<SoftmaxBackward0>)


In [8]:
print(base_model.forward.__doc__)


        Args:
            input_ids: Tensor of gene indices, shape [batch_size, seq_len]
            values: Tensor of expression values, shape [batch_size, seq_len]
            attention_mask: Optional mask tensor, shape [batch_size, seq_len]
            output_cell_emb: Whether to output cell embeddings

        Returns:
            Dictionary containing:
                - 'pred': Predicted expression values
                - 'cell_emb': Cell embeddings (if output_cell_emb=True)
                - 'zero_probs': Zero probabilities (if config.explicit_zero_prob=True)
        


In [38]:
print("Label:", label)
print("Shape:", label.shape)
print("Dtype:", label.dtype)

Label: tensor([4], device='cuda:0')
Shape: torch.Size([1])
Dtype: torch.int64


In [79]:
print(base_model)

ScGPTModel(
  (gene_encoder): ModuleDict(
    (embedding): Embedding(60697, 512, padding_idx=0)
    (enc_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
  )
  (value_encoder): ModuleDict(
    (linear1): Linear(in_features=1, out_features=512, bias=True)
    (linear2): Linear(in_features=512, out_features=512, bias=True)
    (norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.0, inplace=False)
  )
  (transformer): TransformerEncoder(
    (layers): ModuleList(
      (0-11): 12 x TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=512, out_features=512, bias=True)
        )
        (linear1): Linear(in_features=512, out_features=512, bias=True)
        (dropout): Dropout(p=0.0, inplace=False)
        (linear2): Linear(in_features=512, out_features=512, bias=True)
        (norm1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((